## This tool takes the JSON file of tracks view and input to pass each node into the traversal graph algrithm and have the Set of rules to define if the edges are to update or to stay the same in traverse graph algorithm returning True or False.

    IF return True: means that the associated links of the respective node need to be transcript with "_D"

    IF return False: means that the node needs to stay but the link with the next node may need to be changed if the next node was returned as True.
    
    

After the nodes passed through PrecessNode in the traverse code need to deal with the information of:
    creating new links; 
    leave them as they are 
    or transcript them. 
    
    For these above we need to do the following in the traverse code:

Using the function ProcessNode will return if node is False or True
  Which need to be added into Class Link link.processed

And will need also to create new links between the processed nodes with False 
and after need to be added a new link if next node is processd as True updating the following:
         into link.IDl ; 
         link.source;
         link.target

Also need to transcript with "_D" the nodes which link.processed are True 
  adding to:
          link.IDl;
          link.source;
          link.target



In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [11]:
                                '''Location of Json file for input propose'''

path = r'C:\Users\Fonse\Documents\cogitare\12-Fixing_irregularities\nv-tracks-GWR_V1_Barbara-Gyalpo-Nasir.json'
# path = r'C:\Work-DRIVE\dfb-intl-100920\drive\assets\data\ds\c2c-tracks.json'

In [12]:
%pwd

'C:\\Users\\Fonse\\Documents\\cogitare\\12-Fixing_irregularities'

In [13]:
                    '''Creating class for Links and Nodes to be easy to iterate through the data'''

class Link:
    # new_pair type is BerthPair
    # berth_pairs type is [BerthPair] 
    def __init__(self, IDl, source, target):
        self.IDl = IDl
        self.source = source
        self.target = target
        self.processed = False # if Link.processed means that was marked as False and to remain in the data
        
    def __str__(self):   
        return self.source + ':' + self.target
    
    def to_string(self):
        return self.source + ':' + self.target


class Node:
    def __init__(self, ID, x, y, name):
        self.ID = ID
        self.x = x
        self.y = y
        self.name = name
        self.links = []
        self.processed = False
        
    def __str__(self):
        strLinks = ''
        for link in self.links:
            strLinks += ',' + link.__str__()
            print (strLinks)
        
        return self.ID + ',' + str(self.x) + ',' + str(self.y) + ',' + self.name + ':' + strLinks# graphical representation
        
    def AddLink(self, link):
        self.links.append(link)
        


In [14]:
                     '''Function to import the JSON file and using class Node to extract data'''
    
    
new_node= []
def create_pair_nodes(path):
    with open(path) as data_file:    
        data = json.load(data_file) 
    df = pd.DataFrame(data['nodes']) 
 

    
    list_nodes = []
    
    list_node = (list(zip(df['id'],df['x'],df['y'],df['name'])))
    for nod in list_node:
        ID = nod[0]
        x = nod[1]
        y = nod[2]
        name = nod[3]
        instance = Node(ID,x,y,name)
                
        list_nodes.append(instance)
           
    return list_nodes

new_node = create_pair_nodes(path)


In [15]:
dic_nodes = {} # creating dicionary

for node in new_node:
#     print (node.ID)
    dic_nodes.update({node.ID:node})

dic_links = {} # this dictionary is to be created inside function create_pair_source_target 
               # to add just source and target that are not duplicated even if ID is different

'''Function to import the JSON file and using class Link to extract data'''

list_new_link= []

def create_pair_links(path): #list_links
    with open(path) as data_file:    
        data = json.load(data_file)
        
    df = pd.DataFrame(data['links']) 
    
    list_links  = (list(zip(df['id'],df['source'],df['target'])))
    
    list_pair = []
    for link in list_links:
        ids = link[0]
        source = link[1]
        target = link[2]
        concat = source + target
        instance = Link(ids,source,target)
        
#         if '4831_oldoa' == source or '4831_oldoa' == target:
#             print (values)
#         print (link)
        if source in dic_nodes and target in dic_nodes:
        
            if concat in dic_links:
                continue
#             if concat not in dic_links:
    #             dic_links.update({ids:ids,source:source,target:target})
           
#                 print (instance)
            dic_links.update({concat:link})

#             print (instance)
            list_pair.append(instance)
                 
    return (list_pair)
      
list_new_link = create_pair_links(path)

In [16]:
for values in list_new_link:
    if '4831_oldoa' == values.source or '4831_oldoa' == values.target:
        print (values)

986_oldoa:4831_oldoa


In [17]:
                      ''' Adding into class Node list link ( to know which links each node have )'''

for link in list_new_link:

    sourceNode = dic_nodes.get(link.source) # input data into dicionary
    targetNode = dic_nodes.get(link.target) # input data into dicionary

    if sourceNode != None:
        sourceNode.AddLink(link)

    if targetNode != None:
        targetNode.AddLink(link)


In [21]:
'''----------------------------------Rules to apply into traverse graph algorithm-------------------------'''



def ProcessNode(node):
#     print (node.ID + '################################')
    
    if isSig(node): # find if is a signal or a junction and if yes return False (this need to be updated in the class Link for link.processed)
        
        return False
        
    
    if isJunct(node):
      
        return False
    
    nextNode = getNextNode(node) # find if the next node by look if next node is the target
    if nextNode == None: # doesn't have any conection with a signal or junction = is a terminal so is marked as False do stay in the data
        
        return False
    
    prevNode = getPrevNode(node) # find if the next node by looking to the source and to the target
    if prevNode == None:
     
        return "ERROR!"
    
    nextJuctOrSig = searchForJunctOrSig(nextNode, node.ID) # TODO: nextJuctOrNode == nextNode? - look for a signal or node until find one
    if nextJuctOrSig == None: # for termminals

        return False
    
    if node.ID == '89880_didct':

        
        
    # junction to junction different y value
    if not yInThreshouldForJuncToJunc(prevNode.y, nextJuctOrSig.y) and isJunct(prevNode) and isJunct(nextJuctOrSig):# or (isJunct(prevNode) and not isJunct(nextNode)): 
        print ('between junctions' + ' : ' + node.ID)
        return True
    
#     if yInThreshould(prevNode.y, nextJuctOrSig.y) and isSig(prevNode) and isSig(nextJuctOrSig): 
#         return True
   
    
    elif yInThreshould(prevNode.y, nextJuctOrSig.y) and not isSigOrJunct(nextNode) : # if same previous node and next node have the y value between the threshold return True
        
        return True
        
    
    return not isSigOrJunct(prevNode) and not isSigOrJunct(nextNode) # if previous node and next node is not signal or junction return True

    
    
def searchForJunctOrSig(node, prevNodeID): # node is the next node and is our main node from input (nextJuctOrSig)
    if isSigOrJunct(node): # if signal or junction return node
        return node
    
    for link in node.links:
        if link.source == prevNodeID or link.target == prevNodeID:
            continue
        
        if link.source != node.ID:
            return searchForJunctOrSig(dic_nodes.get(link.source), node.ID) # using recursion method to find next node
        else:
            return searchForJunctOrSig(dic_nodes.get(link.target), node.ID) # using recursion method to find next node
    
    return None
    

def getNextNode(node): # this function is to find next node of the main node
    for link in node.links:
        if link.processed: # if Link.processed means that was marked as False and to remain in the data
            continue
        if link.source == node.ID:
            return dic_nodes.get(link.target)
        else:
            return dic_nodes.get(link.source)
    return None


def getPrevNode(node): # this function is to find previous node of the main node
    for link in node.links:
        if not link.processed:
            continue
        if link.source == node.ID:
            return dic_nodes.get(link.target)
        else:
            return dic_nodes.get(link.source)
    return None
    

'''Function to find is a node is a signal or a junction'''
def isSigOrJunct(node): # find if node is a signal or a junction
    
    return isSig(node) or isJunct(node)

def isSig(node): # find if node is signal

    return node.name != ''

def isJunct(node): # find if node is junction

    return len(node.links) > 2 and node.name == ''


def yInThreshould(y1, y2):
 
    return abs(y1-y2) <= 100 # abs convert the result in positive if negative, to be able to check positive and negative values


def yInThreshouldForJuncToJunc(y1, y2):
    return abs(y1-y2) <= 1  # abs convert the result in positive if negative, to be able to check positive and negative values

IndentationError: expected an indented block (<ipython-input-21-f4da937c1066>, line 37)

In [22]:
'''----------------------------------Rules to apply into traverse graph algorithm-------------------------'''


def getNextNodeByLink(link, nodeID): 
    if link.processed:
        return None
    if link.source == nodeID:
        return dic_nodes.get(link.target)
    else:
        return dic_nodes.get(link.source)
    return None

def getNewNeighbour(node, nodeID2): # this function is to find next node of the main node
    for link in node.links:
        if link.processed: # if Link.processed means that was marked as False and to remain in the data
            continue
        if link.source == node.ID and link.target != nodeID2:
            return dic_nodes.get(link.target)
        else:
            return dic_nodes.get(link.source)
    return None

def updateLink(link, nodeIDToStay, newNodeID):
    if nodeIDToStay == link.source:
        link.target = newNodeID
    else:
        link.source = newNodeID




def getNextNodeAndLink(node): # this function is to find next node of the main node
    for link in node.links:
        if link.processed: # if Link.processed means that was marked as False and to remain in the data
            continue
        if link.source == node.ID:
            return (dic_nodes.get(link.target), link)
        else:
            return (dic_nodes.get(link.source), link)
    return (None, None)


def getPrevNodeAndLink(node): # this function is to find previous node of the main node
    for link in node.links:
        if not link.processed:
            continue
        if link.source == node.ID:
            return (dic_nodes.get(link.target), link)
        else:
            return (dic_nodes.get(link.source), link)
    return (None, None)



def replaceLinkTo(links, newLink, nodeID):
    """search in the given list of links for the one linked to nodeID, and replaces it by newLink,
    returning the deleted link."""
    n = 0
    for link in links:
        if link.source == nodeID or link.target == nodeID:
            links[n] = newLink
            return link
        n += 1
    return None

def TraverseGraph(node):
#     print(node.ID+"--") # TODO: delete
    
    if node.processed:
        return
    
    toDelete = ProcessNode(node)
    if toDelete: # aqui so entra um node se tiver next / (assume se for True e o mesmo k dizer toDelete == True)
        nextNode = getNextNode(node)
        print("next:"+nextNode.ID)
        if nextNode == None:
            print("THIS SHOULDN'T HAPPEN2!")
            return
        (prevNode, link) = getPrevNodeAndLink(node)
        if nextNode == (None, None):
            print("THIS SHOULDN'T HAPPEN3!")
            return
        updateLink(link, prevNode.ID, nextNode.ID)
        toTranscript = replaceLinkTo(nextNode.links, link, node.ID)
        toTranscript.source += "_D"
        toTranscript.target += "_D"
        toTranscript.IDl += "_D"
        TraverseGraph(nextNode)
        return
    
    node.processed = True
    for link in node.links:
        neighbour = getNextNodeByLink(link, node.ID)
        if neighbour == None:
            continue

        link.processed = True # to mark node
        
        TraverseGraph(neighbour)
            

In [23]:
          ''' Functions to support function TraverseGraph and the respective TraverseGraph function'''


currentNode = dic_nodes.get('4937_gw103p')
TraverseGraph(currentNode)



In [24]:
'''create lists with ID, source and target for eanch link to after extract to JSON file'''

list_new_id_links = []
list_new_source = []
list_new_target = []

for link in list_new_link:
    
    ID = link.IDl
    source = link.source
    target = link.target
    
    list_new_id_links.append(ID)
    list_new_source.append(source)
    list_new_target.append(target)
    
print (len(list_new_id_links))
    


11341


In [25]:
'''creating a data frame with all new links'''

df_final_links = pd.DataFrame(columns=['id','source','target'])
df_final_links['id'] = list_new_id_links
df_final_links['source'] = list_new_source
df_final_links['target'] = list_new_target

In [26]:
df_final_links.head()

,id,source,target
0,P1_gw103p,1_gw103p,1308_gw103p
1,P2_gw103p,2_gw103p,12_gw103p
2,P3_gw103p,3_gw103p,9527_gw103p
3,P4_gw103p_D,4_gw103p_D,71_gw103p_D
4,P5_gw103p,5_gw103p,89357_gw103p


In [14]:
'''Extract data frame: df_final_links into JSON file'''

# df_final_links.to_json(r'C:\Users\Fonse\Documents\cogitare\Fixing_irregularities\new_links_v8.json', orient='records')